In [167]:
import pandas as pd
import numpy as np

In [168]:
# Import data
file = './purchase_data.csv'
data = pd.read_csv(file)

In [169]:
# Create some useful dataframes to filter data by gender
male = data[data["Gender"] == 'Male']
female = data[data["Gender"] == 'Female']
uk = data[(data["Gender"] != 'Male') & (data["Gender"] != 'Female')]

<h2>Player Count</h2>

In [170]:
# Get a count of unique screennames 
pd.DataFrame({"Total Players": [data.SN.nunique()]})

,Total Players
0,576


<h2>Purchasing Analysis (Total)</h2>

In [171]:
# Create dataframe that holds counts of unique items, average purchase price, number of purchases, and total rev

uniqueItems = data["Item ID"].nunique()
avgPrice = f'{data["Price"].mean():.2f}'
totPurchases = data["Item ID"].count()
totRevenue = f'{data["Price"].sum():.2f}'

pd.DataFrame({'Number of Unique Items': [uniqueItems],
             'Average Purchase Price': [avgPrice],
             'Total Number of Purchases': [totPurchases],
             'Total Revenue': [totRevenue]})

,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,3.05,183,780,2379.77


<h2>Gender Demographics</h2>

In [172]:
# Create counts of unique players by male, female, and non-disclosed 

totMale = male['SN'].nunique()
totFemale = female['SN'].nunique()
totUK = uk['SN'].nunique()
total = data['SN'].nunique()

pd.DataFrame({'Total Count': [totMale, totFemale, totUK],
             'Percentage': [f'{totMale/total:.2%}',f'{totFemale/total:.2%}',f'{totUK/total:.2%}']},
             index=['Male','Female','Other / Not Disclosed'])

,Percentage,Total Count
Male,84.03%,484
Female,14.06%,81
Other / Not Disclosed,1.91%,11


<h2>Purchasing Analysis (Gender)</h2>

In [173]:
# Count the number of purchases by gender

malePurchases = male['Item ID'].count()
femalePurchases = female['Item ID'].count()
ukPurchases = uk['Item ID'].count()

# Calculate the average price of purchase by gender

maleAvgPrice = f'{male["Price"].mean():.2f}'
femaleAvgPrice = f'{female["Price"].mean():.2f}'
ukAvgPrice = f'{uk["Price"].mean():.2f}'

# Add the purchases to calculate total sum by gender

maleTotPrice = f'{male["Price"].sum():.2f}'
femaleTotPrice = f'{female["Price"].sum():.2f}'
ukTotPrice = f'{uk["Price"].sum():.2f}'

# Groupby screenname to sum individual player total purchases (by gender), then take the mean of the individual sums
# This gives us the average total purchase per player, broken down by gender

malePerAvgPrice = f'{male.groupby(["SN"])["Price"].sum().mean():.2f}'
femalePerAvgPrice = f'{female.groupby(["SN"])["Price"].sum().mean():.2f}'
ukPerAvgPrice = f'{uk.groupby(["SN"])["Price"].sum().mean():.2f}'

# Create a df holding this data

pd.DataFrame({'Purchase Count':[malePurchases,femalePurchases,ukPurchases],
             'Average Purchase Price': [maleAvgPrice,femaleAvgPrice,ukAvgPrice],
             'Total Purchase Value': [maleTotPrice,femaleTotPrice,ukTotPrice],
             'Average Total Purchase per Person': [malePerAvgPrice,femalePerAvgPrice,ukPerAvgPrice]},
            index=['Male','Female','Other/Not Disclosed'])

,Average Purchase Price,Average Total Purchase per Person,Purchase Count,Total Purchase Value
Male,3.02,4.07,652,1967.64
Female,3.20,4.47,113,361.94
Other/Not Disclosed,3.35,4.56,15,50.19


<h2>Age Demographics</h2>

In [174]:
# Create bins using np.linspace and adding boundaries of 0 and 100. Also create labels

bins = np.linspace(9,39,num=7)
bins = np.insert(bins,0,0)
bins = np.append(bins,100)
labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Use created bins/labels to create a corresponding 'Age Ranges' column in our original df

data['Age Ranges'] = pd.cut(data['Age'],bins=bins,labels=labels)

# Groupby our new Age Ranges column to calculate age group stats

purchaseCounts = data.groupby(['Age Ranges'])['Item ID'].count()
#averagePrice = data.groupby(['Age Ranges'])['Price'].mean().map(lambda x: f'$ {x:.2f}')
averagePrice = data.groupby(['Age Ranges'])['Price'].mean().apply('{0:,.2f}'.format)
totalRevenue = data.groupby(['Age Ranges'])['Price'].sum().apply('{:,.2f}'.format)

# This last one is a little tricky. Groupby both Age Range and SN to sum individual purchases w/i each
# age range. Then groupby age range once again so we can aggregate via mean() on the ranges.

averagePerPerson = data.groupby(['Age Ranges','SN'])['Price'].sum().groupby(['Age Ranges']).mean() \
                        .apply('{:,.2f}'.format)

# Create the df

pd.DataFrame({'Purchase Count':purchaseCounts,
             'Average Purchase Price':averagePrice,
             'Total Purchase Value':totalRevenue,
             'Avg Total Purchase Per Person': averagePerPerson})

,Average Purchase Price,Avg Total Purchase Per Person,Purchase Count,Total Purchase Value
Age Ranges,,,,
<10,3.35,4.54,23,77.13
10-14,2.96,3.76,28,82.78
15-19,3.04,3.86,136,412.89
20-24,3.05,4.32,365,"1,114.06"
25-29,2.90,3.81,101,293.00
30-34,2.93,4.12,73,214.00
35-39,3.60,4.76,41,147.67
40+,2.94,3.19,13,38.24


<h2>Top Spenders</h2>

In [186]:
# Aggregate total money spent and count of purchases, then use those series to calculate average purchase per SN

totalSpent = data.groupby(['SN'])['Price'].sum()
numberPurchases = data.groupby(['SN'])['Price'].count()
avgPer = (totalSpent/numberPurchases).apply('{0:,.2f}'.format)

# Create the df, sort it by total purchased (descending), and show top 5 via head()

pd.DataFrame({'Total Purchase Value': totalSpent,
             'Purchase Count': numberPurchases,
             'Average Purchase Price': avgPer}).sort_values(by=['Total Purchase Value'],ascending=False).head()

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Lisosia93,3.79,5,18.96
Idastidru52,3.86,4,15.45
Chamjask73,4.61,3,13.83
Iral74,3.40,4,13.62
Iskadarya95,4.37,3,13.10


<h2>Most Popular Items</h2>

In [199]:
# Aggregate total money spent on each item, purchase counts, and include item price

purchaseCount = data.groupby(['Item ID','Item Name'])['Purchase ID'].count()
itemPrice = data.groupby(['Item ID','Item Name'])['Price'].mean().apply('$ {:.2f}'.format)
totRev = data.groupby(['Item ID','Item Name'])['Price'].sum()

# Create df, sort by the purchase count in descending order, and show top 5 via head()

pd.DataFrame({'Purchase Count': purchaseCount,
             'Item Price': itemPrice,
             'Total Purchase Value': totRev}).sort_values(by=['Purchase Count'],ascending=False).head()

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$ 4.23,12,50.76
145,Fiery Glass Crusader,$ 4.58,9,41.22
108,"Extraction, Quickblade Of Trembling Hands",$ 3.53,9,31.77
82,Nirvana,$ 4.90,9,44.10
19,"Pursuit, Cudgel of Necromancy",$ 1.02,8,8.16


<h2>Most Profitable Items</h2>

In [197]:
pd.DataFrame({'Purchase Count': purchaseCount,
             'Item Price': itemPrice,
             'Total Purchase Value': totRev}).sort_values(by=['Total Purchase Value'],ascending=False).head()

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,12,50.76
82,Nirvana,4.90,9,44.10
145,Fiery Glass Crusader,4.58,9,41.22
92,Final Critic,4.88,8,39.04
103,Singed Scalpel,4.35,8,34.80


In [187]:
data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Ranges
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
